# Regression with Fourier Features - Part 3

Apply the space-time Fourier embedded net on the original linear oscillator problem.

In [1]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy
from collections import OrderedDict

# set random seeds
np.random.seed(10)
torch.manual_seed(10);
torch.set_default_dtype(torch.float64)

import numpy as np    
import matplotlib.pyplot as plt

import time
import pylab as pl
from IPython import display
from IPython.display import clear_output

# import neural nets
from PINN.utils.dnn import *

In [2]:
# modify dataset
full_data_path = "../data/LinearOscillator/OU_Noise_energy.mat"
data = scipy.io.loadmat(full_data_path)
# subsample factor
space_factor = 5
time_factor = 5
new_pmc = (data["v_density"].T)[0:-1:time_factor, 0:-1:space_factor]
new_xgrid = data["xi"][:, 0:-1:space_factor]
new_tgrid = data["tspan"][:, 0:-1:time_factor]
# save new data
new_data_path = "../data/LinearOscillator/OU_Noise_energy_subsample{}.mat".format(int(space_factor*time_factor))
scipy.io.savemat(
    new_data_path, {"pmc": new_pmc, "xgrid": new_xgrid, "tgrid": new_tgrid}
)
# test saved data
new_data_path = "../data/LinearOscillator/OU_Noise_energy_subsample25.mat"
data = scipy.io.loadmat(new_data_path)
print(data["pmc"].shape, data["xgrid"].shape, data["tgrid"].shape)
# set random seeds
np.random.seed(10)
torch.manual_seed(10);

data_path = "../data/LinearOscillator/OU_Noise_energy_subsample25.mat"
data = scipy.io.loadmat(data_path)

(1000, 1000) (1, 1000) (1, 1000)


In [3]:
X = cartesian_data(torch.tensor(new_tgrid[0].flatten()), torch.tensor(new_xgrid.flatten()))
y = torch.tensor(new_pmc).T.flatten().reshape(-1, 1)

# normalize data
normalize = True
if normalize:
    X = X / X.mean(dim=0)
    y = y / y.mean()

In [4]:
# define training function
def train(inputs, outputs, model, optim, scheduler, batch_size, epochs, shuffle=True):
    X, y = inputs, outputs
    nx = X.shape[0]
    num_batches = int(nx/batch_size)
    for i in range(epochs):
        print("============================================================\n")
        print("Epoch = {}\n".format(i+1));
        print("============================================================\n")
        model.train()
        if shuffle:
            tmp = np.random.permutation(nx)
            X, y = X[tmp, :].data.clone(), y[tmp, :].data.clone()
        for idx in range(num_batches):
            if idx % 100 == 0:
                print("| => | Batch {} |\n".format(idx+1))
        # closure definition
            def closure():
                optim.zero_grad()
                start_idx = idx*batch_size
                end_idx = (idx+1)*batch_size
                if idx + 1 == num_batches:
                    # if last batch
                    end_idx = -1
                Xb, yb = X[start_idx:end_idx, :].data.clone(), y[start_idx:end_idx, :].data.clone()

                # require gradients
                Xb.requires_grad = True
                # make a prediction on the batch
                y_pred = model.forward(Xb)
                # compute L^2 loss
                loss = torch.mean((y_pred - yb)**2)
                # backpropagate
                loss.backward()
                if idx % 100 == 0:
                    print("==> Batch {} loss = {}".format(idx, loss.item()))
                return loss
            optim.step(closure=closure)
        if scheduler:
            # step scheduler after epoch if there is one
            scheduler.step()
            print("---------- \n")
            print("++ Learning rate reduced, now at = {}".format(scheduler.get_last_lr()[0]))

In [ ]:
# test fourier net in 2d with cartesian product features
# testing Fourier embedded net in 2d (old architecture)
nn_fourier2d_cartesian = FourierProductEmbeddedDNN2d(
    layers_time=[40, 200, 200, 200, 1], 
    layers_space=[40, 200, 200, 200, 1], 
    activation=torch.nn.Tanh, 
    last_layer_activation=None, 
    mt=20, 
    mx=20, 
    freq_stds_t=[1.,2.,10.,20.,50.], 
    freq_stds_x=[1.,2.,10.,20.,50.]
)
optim = torch.optim.Adam(
    nn_fourier2d_cartesian.parameters(),
    lr=8e-3
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9999)
train(X, y, nn_fourier2d_cartesian, optim, scheduler, 2**12, 50, shuffle=True)


Epoch = 1


| => | Batch 1 |

==> Batch 0 loss = 83.77051796436106
| => | Batch 101 |

==> Batch 100 loss = 15.73335460667412
| => | Batch 201 |

==> Batch 200 loss = 10.682262152402046
---------- 

++ Learning rate reduced, now at = 0.0079992

Epoch = 2


| => | Batch 1 |

==> Batch 0 loss = 10.10195692417139
| => | Batch 101 |

==> Batch 100 loss = 6.3549318917203985
| => | Batch 201 |

==> Batch 200 loss = 10.331792184922623
---------- 

++ Learning rate reduced, now at = 0.007998400079999999

Epoch = 3


| => | Batch 1 |

==> Batch 0 loss = 6.586610816842575
| => | Batch 101 |

==> Batch 100 loss = 7.71401230630178
| => | Batch 201 |

==> Batch 200 loss = 6.479502028775772
---------- 

++ Learning rate reduced, now at = 0.007997600239991999

Epoch = 4


| => | Batch 1 |

==> Batch 0 loss = 6.863020650345611
| => | Batch 101 |

==> Batch 100 loss = 6.76255361825474
| => | Batch 201 |

==> Batch 200 loss = 9.23051477704309
---------- 

++ Learning rate reduced, now at = 0.0079968004

In [ ]:
p_pred = nn_fourier2d_cartesian(X).reshape([len(new_xgrid[0]), len(new_tgrid[0])]).detach().numpy().T

In [ ]:
p_exact = data["pmc"]

In [ ]:
idx = 108
plt.figure(1);
plt.plot(data["xgrid"].flatten(), p_pred[idx, :]);
#plt.plot(data["xgrid"].flatten(), p_exact[idx, :]);

In [ ]:
|